In [1]:
import numpy as np
import pandas as pd
import os, csv, math

In [2]:
def city_dict_old(data, city_dict):

#    create a dict of mansion data (1975 - 2010) for each city

    data_list = list(data)
    new_dict = {}
    for idx in range(0,len(data_list)):
        address = data_list[idx][19]
        address = address.split(' ')
        
        if len(address) > 2:
            address = "".join([address[0],address[1]])
        else:
            citycode = str(int(data_list[idx][69]))
            if citycode in city_dict:
                address = city_dict[citycode]
            else:
                address = "other"
        
        new_dict.setdefault(address, []).append(data_list[idx])
    return new_dict

In [3]:
def city_dict_new(data):

#    create a dict of mansion data (1995 - 2015) for each city

    data_list = list(data)
    new_dict = {}
    for idx in range(0,len(data_list)):
        address = data_list[idx][3]
        address = address.split(' ')
        if len(address) > 2:
            address = "".join([address[0],address[1]])
        else:
            address = "other"

        new_dict.setdefault(address, []).append(data_list[idx])
    return new_dict

In [4]:
def build_clean_old(data):

# cleanse old data: make several same 'building name' lines into one
# choose maximum value in ['総全戸数', '総敷地面積', '総延べ面積'] 
# index = [1, 2, 3]

    data_list = list(data)
    ori_id = data_list[0][16]
    #print(data_list[0])
    new_data_list = []
    temp_list = []
    for idx in range(0,len(data_list)):
        
        new_id = data_list[idx][16]
        #print(ori_id, new_id)
        #time.sleep(10)
        #print(temp_list, ori_id, new_id)
        if ori_id == new_id:
            temp_list.append(data_list[idx])
        else:
            temp_arr = np.array(temp_list)
            #print(temp_list, ori_id, new_id)
            processed_list = [temp_list[0][0]]
            processed_list.extend(np.max(temp_arr[:,1:4],axis=0))
            processed_list.extend(temp_list[0][4:16])
            processed_list.extend([ori_id])
            processed_list.extend(temp_list[0][17:])
            new_data_list.append(processed_list)
            ori_id = new_id
            temp_list = []
            temp_list.append(data_list[idx])
        if idx == len(data_list)-1:
            temp_arr = np.array(temp_list)
            processed_list = [temp_list[0][0]]
            processed_list.extend(np.max(temp_arr[:,1:4],axis=0))
            processed_list.extend(temp_list[0][4:16])
            processed_list.extend([ori_id])
            processed_list.extend(temp_list[0][17:])
            new_data_list.append(processed_list)            
    return new_data_list 

In [5]:
def build_clean_new(data):

# cleanse old data: make several same 'building name' lines into one
# choose maximum value in ['全戸数', '総敷地面積', '総延べ面積'] 
# index = [24, 15, 18]

    data_list = list(data)
    ori_id = data_list[0][1]
    new_data_list = []
    temp_list = []
    for idx in range(0,len(data_list)):
        new_id = data_list[idx][1]
        if ori_id == new_id:
            temp_list.append(data_list[idx])
        else:
            temp_arr = np.array(temp_list)
            processed_list = [temp_list[0][0]]
            processed_list.extend([ori_id])
            processed_list.extend(temp_list[0][2:15])
            processed_list.append(np.max(temp_arr[:,15],axis=0))
            processed_list.extend(temp_list[0][16:18])
            processed_list.append(np.max(temp_arr[:,18],axis=0))
            processed_list.extend(temp_list[0][19:24])
            processed_list.append(np.max(temp_arr[:,24],axis=0))
            processed_list.extend(temp_list[0][25:])
            new_data_list.append(processed_list)
            ori_id = new_id
            temp_list = []
            temp_list.append(data_list[idx])
        if idx == len(data_list)-1:
            temp_arr = np.array(temp_list)
            processed_list = [temp_list[0][0]]
            processed_list.extend([ori_id])
            processed_list.extend(temp_list[0][2:15])
            processed_list.append(np.max(temp_arr[:,15],axis=0))
            processed_list.extend(temp_list[0][16:18])
            processed_list.append(np.max(temp_arr[:,18],axis=0))
            processed_list.extend(temp_list[0][19:24])
            processed_list.append(np.max(temp_arr[:,24],axis=0))
            processed_list.extend(temp_list[0][25:])
            new_data_list.append(processed_list)            
    return new_data_list

In [6]:
def load_citylist(filepath):

    data = {}
    with open(filepath,"r") as f:
        reader = csv.reader(f)
        header = next(reader)
    
        for row in reader:
            citycode = str(row[0])
            cityname = str(row[1]) + str(row[4]) + str(row[5])
            data[citycode] = cityname
    return data

In [7]:
fldr = '/Users/xianshiyuming/microbase/Data/customer/Recruit/NameAggregation'
build_old_name = '%s/building_1975-2010.csv' % (fldr)
build_new_name = '%s/building_1995-2015.csv' % (fldr)

city_file = '%s/citylist.csv' % (fldr)
city_dict = load_citylist(city_file)

In [8]:
city_dict['12211']

'千葉県成田市'

In [9]:
build_old = pd.read_csv(build_old_name)
build_new = pd.read_csv(build_new_name)

/home/kaku/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (6,15,17,18,20,21,26,27,28,39,40,44,48,66,76,77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/kaku/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
# change header
# 'マッチ用物件名' and '物件名' into 'building_name'  
build_old_header = list(build_old.columns)
new_build_old_header = build_old_header[:16]
new_build_old_header.append('building_name')
new_build_old_header.extend(build_old_header[17:])

build_new_header = list(build_new.columns)
new_build_new_header = [build_new_header[0]]
new_build_new_header.append('building_name')
new_build_new_header.extend(build_new_header[2:])

In [11]:
#dict
new_build_old_ = city_dict_old(build_old.values, city_dict)
new_build_new_ = city_dict_new(build_new.values)

citylist = list(new_build_new_.keys())
citylist_old = list(new_build_old_.keys())
citylist.extend(citylist_old)
citylist = set(citylist)
citylist = sorted(citylist)

In [12]:
for city in citylist:
    #print(city)
    if city in new_build_old_:
        new_build_old_value = new_build_old_[city]
    else:
        new_build_old_value = []

    if city in new_build_new_:
        new_build_new_value = new_build_new_[city]
    else:
        new_build_new_value = []

    build_old_clean = pd.DataFrame(new_build_old_value, columns=new_build_old_header)
    build_new_clean = pd.DataFrame(new_build_new_value, columns=new_build_new_header)
    print(city, len(build_old_clean.index), len(build_new_clean.index))

    # sort build_old
    if len(build_old_clean.index) > 0:
        new_build_old_sorted = build_old_clean.sort_values('building_name')
        new_build_old_value = new_build_old_sorted.values
        new_build_old_value = build_clean_old(new_build_old_value)
        build_old_clean = pd.DataFrame(new_build_old_value, columns=new_build_old_header)
    
    # sort build_new
    if len(build_new_clean.index) > 0:
        new_build_new_sorted = build_new_clean.sort_values('building_name')
        new_build_new_value = new_build_new_sorted.values
        new_build_new_value = build_clean_new(new_build_new_value)
        build_new_clean = pd.DataFrame(new_build_new_value, columns=new_build_new_header)

    overlap = list(build_old_clean.columns & build_new_clean.columns)

#dff = pd.merge(build_old_clean,build_new_clean,on=['building_name'])
#dff = dff.drop_duplicates('発売年月',keep='last')

    dff2 = pd.merge(build_old_clean,build_new_clean, on=['building_name'],how='outer')
    dff2_col = list(dff2.columns)
    dff2_value = dff2.values

    # add the flag of merge level
    # flag[1,2,3,4] means [perfect, non-perfect, only new data, only old data]  
    flag = []
    for idx in range(len(dff2_value)):
        old_key_value = [dff2_value[idx][1],dff2_value[idx][2],dff2_value[idx][3]]
        new_key_value = [dff2_value[idx][104],dff2_value[idx][95],dff2_value[idx][98]]
        
        if (math.isnan(old_key_value[0]) & math.isnan(old_key_value[1]) & math.isnan(old_key_value[2])) | (math.isnan(new_key_value[0]) & math.isnan(new_key_value[1]) & math.isnan(new_key_value[2])):
            if math.isnan(old_key_value[0]) & math.isnan(old_key_value[1]) & math.isnan(old_key_value[2]):
                flag.append(3)
            else:
                flag.append(4)
        else:
            if old_key_value == new_key_value:
                flag.append(1)
            else:
                flag.append(2)
    
    dff2['flag'] = flag

    # export
    result_file = "%s/dev/step1/%s.csv" % (fldr, city)
    dff2.to_csv(result_file)

other 1843 0
千葉県一宮町 1 0
千葉県佐倉市 189 125
千葉県八千代市 238 177
千葉県八街市 4 0
千葉県勝浦市 19 2
千葉県千葉市中央区 525 346
千葉県千葉市稲毛区 350 204
千葉県千葉市緑区 164 64
千葉県千葉市美浜区 516 467
千葉県千葉市花見川区 265 167
千葉県千葉市若葉区 109 42
千葉県印旛郡酒々井町 10 10
千葉県印西市 164 114
千葉県君津市 10 4
千葉県四街道市 36 14
千葉県大網白里市 0 9
千葉県富津市 1 0
千葉県富里市 13 3
千葉県山武市 2 2
千葉県山武郡大網白里町 9 0
千葉県市原市 161 75
千葉県市川市 894 621
千葉県御宿町 16 0
千葉県成田市 85 51
千葉県我孫子市 174 88
千葉県木更津市 17 12
千葉県東金市 18 4
千葉県松戸市 984 521
千葉県柏市 578 464
千葉県流山市 268 235
千葉県浦安市 342 195
千葉県白井市 98 42
千葉県白子町 6 0
千葉県習志野市 316 213
千葉県船橋市 1335 867
千葉県茂原市 10 2
千葉県袖ケ浦市 5 0
千葉県酒々井町 4 0
千葉県野田市 31 7
千葉県銚子市 2 1
千葉県鋸南町 1 0
千葉県鎌ケ谷市 111 57
千葉県長柄町 3 0
千葉県館山市 11 5
千葉県鴨川市 26 0
埼玉県さいたま市中央区 311 260
埼玉県さいたま市北区 356 212
埼玉県さいたま市南区 543 357
埼玉県さいたま市大宮区 308 206
埼玉県さいたま市岩槻区 27 8
埼玉県さいたま市桜区 184 73
埼玉県さいたま市浦和区 644 526
埼玉県さいたま市緑区 158 96
埼玉県さいたま市西区 35 5
埼玉県さいたま市見沼区 150 47
埼玉県ふじみ野市 181 137
埼玉県三芳町 40 0
埼玉県三郷市 190 81
埼玉県上尾市 149 73
埼玉県上里町 1 0
埼玉県久喜市 47 27
埼玉県伊奈町 11 0
埼玉県入間市 221 89
埼玉県入間郡三芳町 26 26
埼玉県八潮市 91 44
埼玉県加須市 9 0
埼玉県北本市 45 7
埼玉県北葛飾郡杉戸町 7 7
埼玉県南埼